# **1. Import Libraries**

In [1]:
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel

In [ ]:
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel

faq_df = pd.read_excel('/content/Question-Projek.xlsx')

# Inisialisasi tokenizer dan model IndoBERT (base model, tanpa classification head)
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = TFAutoModel.from_pretrained("indobenchmark/indobert-base-p1")

# Fungsi untuk generate embedding dari teks (menggunakan mean pooling dari output last hidden states)
def get_embedding(text):
    inputs = tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='tf')
    outputs = model(inputs)
    last_hidden_state = outputs.last_hidden_state  # shape: (1, seq_len, hidden_dim)
    mask = tf.cast(tf.math.not_equal(inputs['input_ids'], 0), tf.float32)  # attention mask
    mask = tf.expand_dims(mask, axis=-1)  # shape (1, seq_len, 1)

    # Mean pooling dengan masking
    summed = tf.reduce_sum(last_hidden_state * mask, axis=1)
    counted = tf.reduce_sum(mask, axis=1)
    embedding = summed / counted

    return embedding[0].numpy()  # konversi ke numpy array, shape (hidden_dim,)

# Generate embedding untuk semua pertanyaan FAQ
faq_embeddings = []
for question in faq_df['Question']:
    emb = get_embedding(str(question))
    faq_embeddings.append(emb)

faq_embeddings = np.array(faq_embeddings)  # shape (jumlah FAQ, hidden_dim)

# Simpan embedding ke file .npy
np.save('faq_embeddings.npy', faq_embeddings)

# Simpan FAQ ke file JSON (question dan answer saja)
faq_list = faq_df[['Question', 'Answer']].to_dict(orient='records')
with open('faq.json', 'w', encoding='utf-8') as f:
    json.dump(faq_list, f, ensure_ascii=False, indent=2)

print("Selesai simpan embedding dan FAQ.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/656M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some layers from the model checkpoint at indobenchmark/indobert-base-p1 were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-base-p1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for prediction

Selesai simpan embedding dan FAQ.


In [ ]:
from google.colab import files
files.download('/content/faq.json')
files.download('faq_embeddings.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>